In [9]:
#package installs
#E:\Python310\python.exe -m pip install --upgrade pip
#!E:\Python310\python.exe -m pip install git+https://github.com/openai/whisper.git soundfile
!E:\Python310\Scripts\pip3.exe install ffmpeg av
#!E:\Python310\python.exe -m pip uninstall torch torchvision torchaudio -y
!E:\Python310\python.exe -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Found existing installation: torch 2.0.1+cu117
Uninstalling torch-2.0.1+cu117:
  Successfully uninstalled torch-2.0.1+cu117
Found existing installation: torchvision 0.15.2+cu117
Uninstalling torchvision-0.15.2+cu117:
  Successfully uninstalled torchvision-0.15.2+cu117
Found existing installation: torchaudio 2.0.2+cu117
Uninstalling torchaudio-2.0.2+cu117:
  Successfully uninstalled torchaudio-2.0.2+cu117


You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.


Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB 667.8 kB/s eta 1:01:16
     ---------------------------------------- 0.0/2.5 GB 550.5 kB/s eta 1:14:19
     ---------------------------------------- 0.0/2.5 GB 585.1 kB/s eta 1:09:56
     ---------------------------------------- 0.0/2.5 GB 756.6 kB/s eta 0:54:05
     ---------------------------------------- 0.0/2.5 GB 901.1 kB/s eta 0:45:24
     ---------------------------------------- 0.0/2.5 GB 1.1 MB/s eta 0:38:09
     ---------------------------------------- 0.0/2.5 GB 1.3 MB/s eta 0:31:09
     ---------------------------------------- 0.0/2.5 GB 1.4 MB/s eta 0:28:51
     ---------------------------------------- 0.0/2.5 GB 1.8 MB/s eta 0:23:17
     ---------------------------------------- 0.0/2.5 GB 2.2 MB/s eta 0:18:36
     ----

In [1]:
#Global variables
import socket
import os
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data\\'
ENV_FOLDER_DATA_PROC = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data_Proc\\'
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

print('ENV_FOLDER_DATA:' + ENV_FOLDER_DATA)
print('ENV_PRATT:' + ENV_PRATT)

# Create folder structure
import os
os.makedirs(ENV_FOLDER_DATA_PROC + 'segmentation', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'targetdf', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'pratt', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'eps', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'clips', exist_ok=True)


ENV_HOSTNAME:JWGamingPC
ENV_FOLDER_DATA:E:\W4732 Computer Vision\Final Paper Data\
ENV_PRATT:


In [2]:
## Video Processing 1 ##
import pandas as pd
import json
import copy
import csv
import moviepy
from moviepy.editor import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import pickle

def process_video_to_clips(filesuf_mp4, recalc = False):
    #get number after # but before space afterwards
    str_epnum_temp = filesuf_mp4.split('#')[1]
    str_epnum = str_epnum_temp.split(' ')[0]
    int_epnum = int(str_epnum)
    
    #generate filepaths
    filepath_mp4 = ENV_FOLDER_DATA + filesuf_mp4
    print('Episode='+ str_epnum + ' at ' + filepath_mp4)
    folderpath_normmp4 = ENV_FOLDER_DATA_PROC + 'norm\\' + str_epnum + '\\'
    os.makedirs(folderpath_normmp4, exist_ok=True)
    filepath_normmp4 = folderpath_normmp4 + str_epnum + '.mp4'

    folderpath_eps = ENV_FOLDER_DATA_PROC + 'eps\\' + str_epnum + '\\'
    os.makedirs(folderpath_eps, exist_ok=True)
    folderpath_clips = ENV_FOLDER_DATA_PROC + 'clips\\' + str_epnum + '\\'
    os.makedirs(folderpath_clips, exist_ok=True)
    folderpath_clipinfo = ENV_FOLDER_DATA_PROC + 'clipinfo\\'
    os.makedirs(folderpath_clipinfo, exist_ok=True)
    folderpath_sourcedf = ENV_FOLDER_DATA_PROC + 'sourcedf\\'
    os.makedirs(folderpath_sourcedf, exist_ok=True)
    #filepath_mp3 = folderpath_eps + str_epnum + '.mp3'
    #filepath_wav = folderpath_eps + str_epnum + '.wav'
    #folderpath_wavsplit = folderpath_eps + 'wavsplit\\'
    #os.makedirs(folderpath_wavsplit, exist_ok=True)
    filepath_segmentation = ENV_FOLDER_DATA_PROC + 'segmentation\\' + str_epnum + '.psv'
    filepath_targetdf = ENV_FOLDER_DATA_PROC + 'targetdf\\' + str_epnum + '.pickle'
    filepath_pratt = ENV_FOLDER_DATA_PROC + 'pratt\\' + str_epnum + '.pickle'
    filepath_speakers = ENV_FOLDER_DATA_PROC + 'speakers.json'

    #all the clips in a video
    filepath_targetclipinfo = folderpath_clipinfo + str_epnum + '.pickle'

    

    #open dict of speakers
    
    json_speakers = {}
    with open(filepath_speakers) as f:
        json_speakers = json.load(f)
    
    #determine speakers to retain
    list_speakers_keep = []
    #skip episodes not included in json_speakers
    if str_epnum not in json_speakers.keys():
        return 0

    for key,val in json_speakers[str_epnum].items():
        if val == 'Joe Rogan':
            continue
        if val == 'Misc':
            continue
        if val == 'Young Jaime':
            continue
        list_speakers_keep.append(key)

    #declare default dictionary
    dict_default = {}
    dict_default['epnum'] = int_epnum
    dict_default['seg'] = 0
    dict_default['sec_start'] = 0.0

    #use cutoffs to split wav file into sections
    #https://stackoverflow.com/questions/51622865/break-up-a-wav-file-by-timestamp

    listdict_data = []
    #read csv file
    with open(filepath_segmentation, newline='\n') as csv_file:
        reader = csv.reader(csv_file, delimiter='|')
        for row in reader:
            dict_temp = {}
            dict_temp['str_segment'] = row[0]
            dict_temp['speaker'] = row[1]
            dict_temp['sec_start'] = row[2]
            dict_temp['sec_end'] = row[3]
            listdict_data.append(dict_temp)
    
    
    #create normalized master video file
    if not os.path.exists(filepath_normmp4):
        clip_total = VideoFileClip(filepath_mp4).without_audio().resize(height=360)
        clip_total = clip_total.fx(vfx.blackwhite)
        clip_total.write_videofile(filepath_normmp4, fps=10)

    #iterate through each segment
    listdict_clips = []
    for val in listdict_data:

        str_segment = val['str_segment']
        int_segment = int(str_segment)
        speaker = val['speaker']
        sec_start = float(val['sec_start']) 
        sec_end = float(val['sec_end'])
        filepath_clip = folderpath_clips + str_segment + '.mp4'
        duration = sec_end - sec_start

        #skip if it's less than 1 second
        if duration < 1.0:
            continue

        #skip data if it's Joe Rogan or Misc
        if speaker not in list_speakers_keep:
            continue

        dict_temp = copy.deepcopy(dict_default)
        dict_temp['seg'] = int_segment
        dict_temp['duration'] = duration
        dict_temp['sec_start'] = sec_start
        dict_temp['sec_end'] = sec_end
        dict_temp['filesuf_clip'] = str_segment + '.mp4'
        dict_temp['filepath_clip'] = filepath_clip

        #create clip
        ffmpeg_extract_subclip(filepath_normmp4, sec_start, sec_end, targetname=filepath_clip)

        #save dictionary
        listdict_clips.append(dict_temp)

    #save data
    
    with open(filepath_targetclipinfo, 'wb') as file:
        pickle.dump(listdict_clips, file, protocol=pickle.HIGHEST_PROTOCOL)
        
    
    #end loop through data
    
    




In [20]:
filesuf_mp4 = 'Joe Rogan Experience #568 - Dr. Rhonda Patrick.mp4'
process_video_to_clips(filesuf_mp4)

Episode=568 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experience #568 - Dr. Rhonda Patrick.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command succes

In [12]:
## Video Processing 1 ##
import pandas as pd
import json
import copy
import csv
import pickle
import torchvision
import itertools
import torch

def process_clips_to_frames(filesuf_mp4, recalc = False):
    #get number after # but before space afterwards
    str_epnum_temp = filesuf_mp4.split('#')[1]
    str_epnum = str_epnum_temp.split(' ')[0]
    int_epnum = int(str_epnum)
    
    #generate filepaths
    filepath_mp4 = ENV_FOLDER_DATA + filesuf_mp4
    print('Episode='+ str_epnum + ' at ' + filepath_mp4)
    folderpath_normmp4 = ENV_FOLDER_DATA_PROC + 'norm\\' + str_epnum + '\\'
    os.makedirs(folderpath_normmp4, exist_ok=True)
    filepath_normmp4 = folderpath_normmp4 + str_epnum + '.mp4'

    #folderpath_eps = ENV_FOLDER_DATA_PROC + 'eps\\' + str_epnum + '\\'
    #os.makedirs(folderpath_eps, exist_ok=True)
    folderpath_clips = ENV_FOLDER_DATA_PROC + 'clips\\' + str_epnum + '\\'
    os.makedirs(folderpath_clips, exist_ok=True)
    folderpath_clipinfo = ENV_FOLDER_DATA_PROC + 'clipinfo\\'
    os.makedirs(folderpath_clipinfo, exist_ok=True)
    folderpath_sourcedf = ENV_FOLDER_DATA_PROC + 'sourcedf\\'
    os.makedirs(folderpath_sourcedf, exist_ok=True)
    filepath_sourcedf = folderpath_sourcedf + str_epnum + '.pickle'
    #filepath_mp3 = folderpath_eps + str_epnum + '.mp3'
    #filepath_wav = folderpath_eps + str_epnum + '.wav'
    #folderpath_wavsplit = folderpath_eps + 'wavsplit\\'
    #os.makedirs(folderpath_wavsplit, exist_ok=True)
    #filepath_segmentation = ENV_FOLDER_DATA_PROC + 'segmentation\\' + str_epnum + '.psv'
    #filepath_targetdf = ENV_FOLDER_DATA_PROC + 'targetdf\\' + str_epnum + '.pickle'
    filepath_pratt = ENV_FOLDER_DATA_PROC + 'pratt\\' + str_epnum + '.pickle'
    filepath_speakers = ENV_FOLDER_DATA_PROC + 'speakers.json'

    #all the clips in a video
    filepath_targetclipinfo = folderpath_clipinfo + str_epnum + '.pickle'

    
    listdict_clips = []
    #read pickle file
    with open(filepath_targetclipinfo, 'rb') as file:
        listdict_clips = pickle.load(file)
    

    #iterate through each segment
    #create the object that will be loaded into the neural network
    dict_sourceframes = {}
    for val in listdict_clips:

        int_segment = val['seg']
        sec_start = float(val['sec_start']) 
        sec_end = float(val['sec_end'])
        filepath_clip = val['filepath_clip'] 
        duration = val['duration']

        # https://pytorch.org/vision/main/generated/torchvision.io.VideoReader.html
        # https://pytorch.org/vision/main/auto_examples/others/plot_video_api.html#sphx-glr-auto-examples-others-plot-video-api-py 
        # if you have an error here, download the latest toolkit:
        # https://developer.nvidia.com/cuda-downloads
        torchvision.set_video_backend('pyav')
        reader = torchvision.io.VideoReader(filepath_clip, "video")
        reader.set_current_stream("video")
        transform = torchvision.transforms.Grayscale()

        video_frames = torch.empty(0)
        frames = [] 
        ct = 1
        for frame in reader:
            tempframe = transform(frame['data'])
            frames.append(tempframe)
            ct += 1
        for x in range(40):
            frames.append( torch.empty((1,360,640), dtype=torch.int8) )
        if len(frames) > 0:
            video_frames = torch.stack(frames[:40], 0)
        
        dict_sourceframes[int_segment] = video_frames

    #save data
    with open(filepath_sourcedf, 'wb') as file:
        pickle.dump(dict_sourceframes, file, protocol=pickle.HIGHEST_PROTOCOL)
    
    #end loop through data

In [13]:
filesuf_mp4 = 'Joe Rogan Experience #568 - Dr. Rhonda Patrick.mp4'
process_clips_to_frames(filesuf_mp4)

Episode=568 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experience #568 - Dr. Rhonda Patrick.mp4


False